In [ ]:
pip install -r requirements.txt

In [8]:
from fastapi import FastAPI, File, UploadFile, Query, HTTPException, Request, Form
from fastapi.responses import JSONResponse, FileResponse
from fastapi.encoders import jsonable_encoder
import shutil
from typing import List
import subprocess
from osllm import OpenSourceLLM
import nest_asyncio
import uvicorn
from pydantic import BaseModel

app = FastAPI()

llm = OpenSourceLLM()
@app.get("/")
def read_root():
    return {"message": "Estoy vivo!"}

class Modelo(BaseModel):
    nombre: str
    descripcion: str

class OpcionesModelo(BaseModel):
    modelos: List[Modelo]

@app.post("/uploadfile/")
async def create_upload_file(file: UploadFile = File(...),speakers: str = Query(..., title="Speaker Name", description="Name of the speaker"),
                             language: str = Query(..., title="Language", description="Language for speech-to-text")):
    try:
        if file.filename.endswith('.mp3'):
            with open(f"/workspace/{file.filename}", "wb") as mp3_file:
                shutil.copyfileobj(file.file, mp3_file)
            #subprocess.run(["python","audio2text.py",f"uploaded_files/{file.filename}",speakers,language])
            return JSONResponse(content=jsonable_encoder({"message": f"File uploaded successfully, speakers= {speakers}, language={language}"}), status_code=200)
        else:
            return JSONResponse(content=jsonable_encoder({"error": "Only MP3 files are allowed"}), status_code=400)
    except Exception as e:
        return JSONResponse(content=jsonable_encoder({"error": str(e)}), status_code=500)

@app.get("/speech-to-text/")
async def speech_to_text(speakers: str = Query(..., title="Speaker Name", description="Name of the speaker"),
                         language: str = Query(..., title="Language", description="Language for speech-to-text")):
    response_content = {
        "message": f"speech-to-text conversion for {speakers} in {language} language."
    }
    return response_content

@app.get("/get-transcript/")
async def get_text_file(name_file: str = Query(..., title="Nombre del archivo", description="Nombre del archivo")):
    file_path = f"/workspace/transcript.txt"
    transcription_processor.run_transcription('/workspace/' + name_file)
    return FileResponse(file_path, filename=f"transcript.txt", media_type="text/plain")

@app.get("/modelos", response_model=OpcionesModelo)
async def obtener_modelos():
    opciones = []
    for nombre, descripcion in OpenSourceLLM.OPCIONES_MODELOS.items():
        opciones.append(Modelo(nombre=nombre, descripcion=descripcion))
    return OpcionesModelo(modelos=opciones)

@app.post("/generar_texto", response_model=str)
async def generar_texto(request: Request, form_data: Form(...)):
    modelo_elegido = form_data.modelo
    texto = form_data.texto

    llm = OpenSourceLLM(modelo_elegido)
    respuesta = llm.text2text(texto)

    return respuesta

nest_asyncio.apply()

config = uvicorn.Config(app=app, host="127.0.0.1", port=8000, loop='asyncio')
server = uvicorn.Server(config)
await server.serve()

Modelos disponibles:
- llama2: Llama2 (70B)
- mixtral: Mixtral (35B)
- bakllava: Bakllava (15B)


FastAPIError: Invalid args for response field! Hint: check that default=Ellipsis extra={} is a valid pydantic field type